<a href="https://colab.research.google.com/github/darkmochalover/DS_TermProject_AL2/blob/main/preprocess_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dictionary

In [34]:
# Data Path
raw_data_path = "/content/drive/MyDrive/DS_TermProject/data/combined_mbti_df.csv"
# data_path = "data/preprocessed_mbti_data.csv"


audio_qualities = [
    'danceability_mean',
    'valence_mean',
    'energy_mean',
    'loudness_mean',
    'acousticness_mean',
    'instrumentalness_mean',
    'liveness_mean',
]

# 장조/단조 (Major/Minor)
all_tones = [
    'Cminor_count', 'CMajor_count', 'C#/Dbminor_count', 'C#/DbMajor_count',
    'DMajor_count', 'D#_EbMajor_count', 'Eminor_count', 'EMajor_count',
    'Fminor_count', 'FMajor_count', 'F#/Gbminor_count', 'GMajor_count',
    'G#/Abminor_count', 'G#/AbMajor_count', 'Aminor_count', 'AMajor_count',
    'A#/Bbminor_count', 'BMajor_count', 'Dminor_count', 'D#_Ebminor_count',
    'Gminor_count', 'A#/BbMajor_count', 'F#/GbMajor_count', 'Bminor_count'
]

major_tones = [
    'CMajor_count', 'C#/DbMajor_count',
    'DMajor_count', 'D#_EbMajor_count', 
    'EMajor_count',
    'FMajor_count', 
    'GMajor_count', 'G#/AbMajor_count', 
    'AMajor_count', 'BMajor_count', 'A#/BbMajor_count', 
    'F#/GbMajor_count'
]
minor_tones = [
    'Cminor_count', 'C#/Dbminor_count', 
    'Eminor_count', 
    'Fminor_count', 'F#/Gbminor_count', 
    'G#/Abminor_count',  
    'Aminor_count', 'A#/Bbminor_count', 
    'Dminor_count', 'D#_Ebminor_count',
    'Gminor_count', 
    'Bminor_count'
]


# 열 이름을 바꾸기 위해 리스트로 저장함
renamed_columns =  [
    'danceability',
    'valence',
    'energy',
    'loudness',
    'acousticness',
    'instrumentalness',
    'liveness'
]


Install WandB

In [35]:
!pip install -qU wandb

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE

import itertools

import wandb
import random

Step 1: Import W&B and Login

In [37]:
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint, WandbEvalCallback

In [38]:
encoder_list = ['LabelEncoder', 'OneHotEncoder']
scaler_list = ['StandardScaler', 'MinMaxScaler', 'RobustScaler', 'No Scale']
test_size_list = [0.3, 0.2, 0.1]
do_smote = ['True', 'False']

In [39]:
config = {
        'encoder' : 'LabelEncoder',
        'scaler' : 'StandardScaler',
        'test_size' : 0.3,
        "do_smote" : True
    }

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="MBTI_playlist",
    
    # track hyperparameters and run metadata
    config = config
)

config=wandb.config


Data Loading

In [40]:
df = pd.read_csv(raw_data_path)

Feature Engineering

In [41]:
# Subset only measures of centers
X = df.iloc[: , :22] # 처음 22개 열을 선택해서 저장
X = df[audio_qualities] # 오디오 품질과 해당 열이 있는 열을 선택해서 저장


categories = renamed_columns[:]
X.columns = renamed_columns


# 장조/단조의 개수의 합을 계산해서 저장 (C장조, D단조, .. 이렇게 따로 계산되는거 말고, 위에 지정된 list 이용해서 sum값 넣어줌)
X['major_count'] = df[major_tones].sum(axis=1).astype('int64')
X['minor_count'] = df[minor_tones].sum(axis=1).astype('int64')

<ipython-input-41-482ca4c8bc7c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['major_count'] = df[major_tones].sum(axis=1).astype('int64')
<ipython-input-41-482ca4c8bc7c>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['minor_count'] = df[minor_tones].sum(axis=1).astype('int64')


## Encoding & Scaling Function

In [42]:
def Encoding(df, encoding_method):
    df = df.copy()

    if(encoding_method == 'LabelEncoder'):
        encoder = LabelEncoder()
        target = encoder.fit_transform(df[['mbti']])
        

    if(encoding_method == 'OneHotEncoder'):
        encoder = OneHotEncoder(sparse=False)
        target = encoder.fit_transform(df[['mbti']])

    return target

def Scaling(scale_method, X_train, X_test):
    if( scale_method == 'No Scale'):
        return X_train, X_test

    elif(scale_method == 'StandardScaler'):
        scaler = StandardScaler()

    elif(scale_method == 'MinMaxScaler'):
        scaler = MinMaxScaler()

    elif(scale_method == 'RobustScaler'):
        scaler = RobustScaler()


    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.fit_transform(X_test)

    return X_train_scaled, X_test_scaled


In [45]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

combinations = list(itertools.product(encoder_list, scaler_list, test_size_list, do_smote))

for encoder, scaler, test_size, do_smote in combinations:
    print(f"Encoder: {encoder}, Scaler: {scaler}, Test Size: {test_size}, Smote Phase: {do_smote}")
    wandb.config.update({
          'encoder' : encoder,
          'scaler' : scaler,
          'test_size' : test_size,
          "do_smote" : do_smote
})

    y = Encoding(df = df, encoding_method=encoder)
    # print(y[:5])

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 5, test_size = test_size)

    X_train, X_test = Scaling(scale_method=scaler, X_train=X_train, X_test=X_test)

    if(do_smote=='True'): # If true,
        smote = SMOTE(sampling_strategy='auto', random_state=0)
        X_train, y_train = smote.fit_resample(X_train,y_train)

    print('Train 피처/레이블 데이터 세트: ', X_train.shape, y_train.shape)

    wandb.log({'Train feature 데이터 세트' : X_train.shape})
    wandb.log({'Train label 데이터 세트' : X_train.shape})

    # Random Forest Model with top 5 features
    rf_model = RandomForestClassifier(random_state=42)
    rf_model.fit(X_train, y_train)
    rf_predictions = rf_model.predict(X_test)
    rf_accuracy = accuracy_score(y_test, rf_predictions)
    print("Random Forest Accuracy with Top 5 Features:", rf_accuracy)
    wandb.log({'Random Forest Accuracy with Top 5 Features' : rf_accuracy})
        


Encoder: LabelEncoder, Scaler: StandardScaler, Test Size: 0.3, Smote Phase: F
Train 피처/레이블 데이터 세트:  (2856, 9) (2856,)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Random Forest Accuracy with Top 5 Features: 0.24571428571428572
Encoder: LabelEncoder, Scaler: StandardScaler, Test Size: 0.2, Smote Phase: F
Train 피처/레이블 데이터 세트:  (3264, 9) (3264,)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Random Forest Accuracy with Top 5 Features: 0.25091799265605874
Encoder: LabelEncoder, Scaler: StandardScaler, Test Size: 0.1, Smote Phase: F
Train 피처/레이블 데이터 세트:  (3672, 9) (3672,)


...

Random Forest Accuracy with Top 5 Features: 0.04
Encoder: OneHotEncoder, Scaler: No Scale, Test Size: 0.2, Smote Phase: F
Train 피처/레이블 데이터 세트:  (3264, 9) (3264, 16)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Random Forest Accuracy with Top 5 Features: 0.046511627906976744
Encoder: OneHotEncoder, Scaler: No Scale, Test Size: 0.1, Smote Phase: F
Train 피처/레이블 데이터 세트:  (3672, 9) (3672, 16)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Random Forest Accuracy with Top 5 Features: 0.0488997555012225


In [46]:
wandb.finish()

Random Forest Accuracy with Top 5 Features,▇██▆▆▆▇▇█▇▇█▂▂▂▁▁▁▂▂▂▂▂▂
Random Forest Accuracy with Top 5 Features,0.0489
